In [1]:
import cadquery as cq

from jupyter_cadquery import (
    versions,
    show, PartGroup, Part, 
    get_viewer, close_viewer, get_viewers, close_viewers, open_viewer, set_defaults, get_defaults, open_viewer,
    get_pick,
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay

enable_replay(False)

set_defaults(
    cad_width=640, 
    height=480, 
)

print()
versions()

Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay

jupyter_cadquery  3.1.0rc4
cad_viewer_widget 1.3.2
OCP               7.5.3


In [2]:
cv = open_viewer("Assembly", cad_width=850, height=1240)
set_defaults(viewer="Assembly")

In [3]:
SWITCH_BORDER_HEIGHT = 5
SWITCH_PLATE_HEIGHT = 1.2
PCB_HEIGHT = 1.6
PCB_BOTTOM_SPACING = 2.4
BOTTOM_PLATE_HEIGHT = 1.2
DISPLAY_SPACING = 8
CASE_HEIGHT = SWITCH_BORDER_HEIGHT\
            + SWITCH_PLATE_HEIGHT\
            + PCB_HEIGHT\
            + PCB_BOTTOM_SPACING\
            + BOTTOM_PLATE_HEIGHT


In [19]:
def createCaseOutline():
    return cq.Workplane("XY")\
    .moveTo(78.77, -42.15)\
    .lineTo(78.77, -90.9)\
    .radiusArc((75.91, -96.0), 8)\
    .lineTo(70.57, -99.08)\
    .radiusArc((68.98, -105.04), -4.3)\
    .lineTo(82.05, -127.54)\
    .radiusArc((88.00, -129.13), -4.3)\
    .radiusArc((118.04, -116.55), 79.9)\
    .lineTo(176.41, -116.55)\
    .radiusArc((179.39, -113.61), -2.9)\
    .lineTo(179.39, -110.34)\
    .radiusArc((182.34, -107.39), 2.9)\
    .lineTo(194.62, -107.39)\
    .radiusArc((197.6, -104.45), -2.9)\
    .lineTo(197.6, -51.11)\
    .radiusArc((194.66, -48.13), -2.9)\
    .lineTo(182.34, -48.14)\
    .radiusArc((179.4, -45.16), 2.9)\
    .lineTo(179.4, -41.12)\
    .radiusArc((177.15, -37.56), -3.7)\
    .radiusArc((101.24, -37.15), -104.25)\
    .lineTo(83.77, -37.15)\
    .radiusArc((78.77, -42.15), -5)\
    .close()

def createBottomPlate():
    return createCaseOutline()\
        .faces()\
        .offset2D(-3)\
        .extrude(BOTTOM_PLATE_HEIGHT)

def createCase():
    return createCaseOutline()\
        .faces()\
        .extrude(CASE_HEIGHT)\
        .fillet(2)\
        .cut(createBottomPlate())

In [5]:
switches = cq.importers.importDXF('switch_outline.dxf', tol=1e-3)\
    .wires()\
    .toPending()\
    .faces()\
    .extrude(BOTTOM_PLATE_HEIGHT + PCB_BOTTOM_SPACING + PCB_HEIGHT + SWITCH_PLATE_HEIGHT)

In [6]:
keycaps = cq.importers.importDXF('keycaps_outline.dxf', tol=1e-3)\
    .wires()\
    .toPending()\
    .faces()\
    .extrude(SWITCH_BORDER_HEIGHT)\
    .translate((0,0,CASE_HEIGHT - SWITCH_BORDER_HEIGHT))

In [7]:
pcb = cq.importers.importDXF('pcb_outline.dxf', tol=1e-3)\
    .wires()\
    .toPending()\
    .faces()\
    .extrude(BOTTOM_PLATE_HEIGHT + PCB_BOTTOM_SPACING + PCB_HEIGHT)

In [20]:
case = createCase()\
    .cut(keycaps)\
    .cut(switches)\
    .cut(pcb)

In [41]:
all = (cq.Assembly()\
       .add(case, name="case", color=cq.Color("paleturquoise3"))\
       .add(createBottomPlate(), name="plate", color=cq.Color("paleturquoise4")))
show(all)

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.02s
